# Introduzione
In questo file troviamo un modello di rete neurale per la classificazione del dataset COVID-CXR4.\
IL dataset è stato scaricato da [Kaggle](https://www.kaggle.com/datasets/andyczhao/covidx-cxr2) e messo dentro la cartella datasets \(questa cartella è ignorata da git perchè il dataset è grosso)\
I modelli salvati si possono trovare sotto la cartella [models](models) in modo da poterli usare senza rifare l'addestramento.

Questo *interactive pyhon notebook* è suddiviso in 3 parti principali:
- **Dataset**: in cui viene caricato, modificato e salvato in una cache l'intero dataset di immagini.
- **Modello**: in cui vengono creati e addestrati l'autoencoder e il classificatore.
- **Contrastive Learning**: in cui viene applicata la tecnica di contrastive learning per migliorare gli embedding da passare al classificatore.

Ogni parte del notebook contiene anche dei grafici e immagini per mostrare come i vari modelli si comportano.

In questa prima parte vengono importati le varie librerie usate e vengono create le variabili globali.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from keras import layers, models, optimizers, ops, backend
from src.functions import datasets, all_models, plots

upperdir = '' # change this if the script is not in the same directory as the parent folder
models_dir = f"{upperdir}models"
datasets_dir = f"{upperdir}datasets"

# Ensure the directories exist
os.makedirs(models_dir, exist_ok=True)
os.makedirs(datasets_dir, exist_ok=True)

# Variabili per i modelli
shape = (224, 224)
latent_space = 128
epochs_autoencoder = 25
epochs_classifier = 100

# Dataset
Modifica e caricamento del dataset.
Il dataset usato in questo caso è il dataset [COVIDx CXR-4](https://www.kaggle.com/datasets/andyczhao/covidx-cxr2).

Le modifiche apportate sono:
- Rimozione dei canali di colore \(alcune immagini hanno per esempio delle scritte rosse\); quindi ogni immagine è in scala di grigio.
- Ridimensionamento a 224x224 \(molte immagini sono 1024x1024 ma ci sono anche di dimensioni diverse\)

Le immagini importate sono sottoforma di array di numpy a 8bit che poi vengono salvate in un file cache (~4GB).\
Il primo blocco di codice carica il dataset dalle funzioni per la modifica del dataset. La funzione `covid_cxr_data` è quella responsabile per il caricamento dei dati.

Inoltre viene mostrato quante classi ci sono e come sono distribuite all'interno del dataset.\
Come si può notare il training set è sbilanciato verso una classe e questo non aiuta per l'addestramento del classificatore.

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = datasets.covid_cxr_data(datasets_dir, shape)
total_classes, class_weights = plots.plot_class_distribution(y_train, y_val, y_test)

# Modello
In questa sezione vediamo i modelli per l'autoencoding e per la classificazione.\
Come già illustrato precedentemente il processo dei modelli è il seguente:
- **Autoencoder** che prende in input delle immagini di grandezza definita nella prima parte di codice (shape), per comprimerle tramite layer convoluzionali fino ad una rappresentazione latente definita anch'essa in testa al codice (latent_space).\
  L'autoencoder a quel punto cercherà di ricostruire l'immagine dalla sua rappresentazione latente tramite la parte di decoder.\
  Il modello verrà valutato tramite *MSE* dato che i valori dei pixel saranno compresi tra \[0,1\]
- **Classifier** che prende in input la rappresentazione latente dell'immagine e la trasforma in probabilità di una o dell'altra classe.\
  Questo modello verrà valutato tramite *sparse_categorical_crossentropy* dato che restituirà un array con le probabilità per ogni classe.

In [ ]:
custom_models = all_models.CustomModels(latent_space, shape, models_dir)

### Autoencoder
Il primo modello creato è l'autoencoder e usa gli stessi principi delle CNN per creare una rappresentazione compatta delle immagini. Infatti il modello è composto da dei Convolutional Layer che, riducono la dimensione spaziale per aumentare la dimensionde dei filtri.\
L'encoder ha inoltre dei layer di BatchNormalization.

Questo modello è quello più lungo da addestrare solamente perchè ha abbastanza parametri e il dataset, essendo grande, non ci sta in memoria.\
Per queste ragioni la batch è piccola e generata da una funzione.

In [ ]:
should_train, autoencoder = custom_models.autoencoder_build()
encoder = autoencoder.get_layer('encoder')

if should_train:
    batch = 32
    batch_steps = len(x_train) // batch
    batch_val_steps = len(x_val) // batch
    gen_train = datasets.data_generator_autoencoder(x_train, batch)
    gen_val = datasets.data_generator_autoencoder(x_val, batch)

    history_auto = autoencoder.fit(gen_train, validation_data=gen_val,
                                epochs=epochs_autoencoder,
                                steps_per_epoch=batch_steps, validation_steps=batch_val_steps)
    custom_models.save_model_and_history(autoencoder, history_auto)

Dopo aver trainato o caricato l'autoencoder facciamo la prediction dell'intero dataset:

In [ ]:
x_train_encoded, x_val_encoded, x_test_encoded = datasets.load_or_predict_values(datasets_dir, encoder, x_train, x_val, x_test, 32)

### Classificatore
Il classificatore è un modello semplice con 2 layer densi e un layer finale per la classificazione con la softmax.\
Essendo i dati molto più piccoli le batch possono essere alte e si possono avere molte più epoche per far imparare.

Purtroppo essendo il dataset molto sbilanciato verso una classe l'addestramento viene influenzato negativamente se non si fanno delle correzioni.

In [ ]:
should_train, classifier = custom_models.classifier_build(total_classes)

if should_train:
    batch = 1024

    batch_steps = len(x_train) // batch
    batch_val_steps = len(x_val) // batch

    history_class = classifier.fit(x_train_encoded, y_train, validation_data=(x_val_encoded, y_val),
                                   epochs=epochs_classifier,
                                   batch_size=batch, class_weight=class_weights)
    custom_models.save_model_and_history(classifier, history_class)

Anche dopo il classificatore, che sia stato caricato o addestrato dobbiamo predire i risultati e salvarli in variabili:

In [ ]:
batch_predictions = 32

y_train_pred = classifier.predict(x_train_encoded)
y_val_pred = classifier.predict(x_val_encoded)
y_test_pred = classifier.predict(x_test_encoded)

### Risultati
Di seguito i risultati dell'addestramento se è stato fatto, altrimenti vengono mostrati solo delle predizioni di alcuni dati di test.

In [ ]:
history = [
    custom_models.load_history_of(autoencoder),
    custom_models.load_history_of(classifier),
]

plots.plot_history(history, ['loss', 'accuracy'])

In [ ]:
datasets_pred = [
    ("Training Data", y_train, y_train_pred),
    ("Validation Data", y_val, y_val_pred),
    ("Test Data", y_test, y_test_pred)
]
plots.plot_confusion_matrix(datasets_pred, total_classes)

In [ ]:
decoder = autoencoder.get_layer('decoder')
plots.plot_autoencoder_predictions(x_test, x_test_encoded, y_test, y_test_pred, decoder)

# Contrastive Learning
Il blocco di codice sottostante definisce un modello siamese che utilizza la loss contrastive per modificare gli embedding in modo da migliorare la classificazione.\
Il modello contrastivo è un semplice modello con due layer densi che produce un output scalare con attivazione sigmoid, rappresentando la probabilità che due input appartengano alla stessa classe.\
Il modello siamese utilizza due torri identiche del modello contrastivo per calcolare la distanza tra due rappresentazioni latenti, utilizzando una funzione Lambda personalizzata per calcolare la distanza.

In [ ]:
def make_pairs_generator(x, y_true, y_pred, batch):
    classes = len(np.unique(y_true))
    correct_predictions = np.where(y_true == y_pred)[0]
    correct_predictions_class = [np.where(y_true[correct_predictions] == i)[0] for i in range(classes)]

    not_correct = np.where(y_true != y_pred)[0]
    yield len(not_correct)

    while True:
        np.random.shuffle(not_correct)
        for i in not_correct:
            pairs_1 = []
            pairs_2 = []
            labels = []

            for _ in range(batch // 2):
                # Positive pair
                x1 = x[i]
                label1 = y_true[i]
                x2 = x[np.random.choice(correct_predictions_class[label1])]
                pairs_1 += [x1]
                pairs_2 += [x2]
                labels += [0]

                # Negative pair
                label2 = np.random.choice(classes)
                while label2 == label1:
                    label2 = np.random.choice(classes)
                idx = np.random.choice(correct_predictions_class[label2])
                x2 = x[idx]
                pairs_1 += [x1]
                pairs_2 += [x2]
                labels += [1]

            yield (np.array(pairs_1), np.array(pairs_2)), np.array(labels)

batch_train_pairs = 32
gen_train_pair = make_pairs_generator(x_train_encoded, y_train, np.argmax(y_train_pred, axis=1), batch_train_pairs)
gen_val_pair = make_pairs_generator(x_val_encoded, y_val, np.argmax(y_val_pred, axis=1), batch_train_pairs)
batch_train_pairs_steps = next(gen_train_pair)
batch_val_pairs_steps = next(gen_val_pair)

print(f"Train pairs: {batch_train_pairs} * {batch_train_pairs_steps}")
print(f"Validation pairs: {batch_train_pairs} * {batch_val_pairs_steps}")

Questo codice implementa due loss functions.\
In particolare per problemi di apprendimento contrastivo, come il confronto tra coppie di campioni vengono usate le seguenti loss:

1. contrastive_loss: calcola la perdita contrastiva standard.\
   Penalizza le coppie di campioni in base alla loro distanza predetta (y_pred) e alla loro etichetta reale (y_true).\
   Se i campioni sono simili (y_true=0), penalizza le distanze grandi.\
   Se i campioni sono diversi (y_true=1), penalizza le distanze piccole.
2. contrastive_SNN_loss: calcola una variante della perdita contrastiva basata su una funzione softmax normalizzata.\
   Utilizza una temperatura (temperature) per controllare la "morbidezza" della penalizzazione.\
   Penalizza le coppie in base alla probabilità relativa di similarità, calcolata come rapporto tra le distanze esponenziali normalizzate.\
   La loss usata è la [Soft Nearest Neighbors Loss](https://lilianweng.github.io/posts/2021-05-31-contrastive/#soft-nearest-neighbors-loss)

In [ ]:
temperature = 1

def distance(vects):
    x, y = vects
    sum_square = ops.sum(ops.square(x - y), axis=1, keepdims=True)
    return ops.sqrt(ops.maximum(sum_square, backend.epsilon()))

def loss(margin=1.0, temperature=1.0):
    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        square_pred = ops.square(y_pred)
        margin_square = ops.square(ops.maximum(margin - (y_pred), 0))
        return ops.mean((1 - y_true) * square_pred + (y_true) * margin_square)

    def contrastive_SNN_loss(y_true, y_pred):
        mask = ops.equal(y_true, 0)
        exp_similarity = ops.exp(ops.negative(y_pred / temperature))

        numerator = ops.sum(exp_similarity * mask)
        denominator = ops.sum(exp_similarity) + backend.epsilon()  # Add epsilon to avoid division by zero

        safe_ratio = numerator / denominator
        safe_ratio = ops.maximum(safe_ratio, backend.epsilon())  # Ensure ratio is not less than epsilon

        return ops.negative(ops.mean(ops.log(safe_ratio)))

    return contrastive_SNN_loss


Il modello e la rete siamese usata per l'addestramento sono i seguenti:

In [ ]:
model_path, history_path =  custom_models.get_full_path('correction')

try:
    correction = models.load_model(model_path, custom_objects={'distance': distance})
    history_siamese = np.load(history_path, allow_pickle=True).item()
except:
    half_space = int(latent_space/2)

    correction_in = layers.Input(shape=(latent_space,))
    x = layers.Dense(half_space, activation='relu')(correction_in)
    correction_out = layers.Dense(latent_space, activation='sigmoid')(x)
    correction = models.Model(correction_in, correction_out, name='correction')

    siamese_in_1 = layers.Input(shape=(latent_space,))
    siamese_in_2 = layers.Input(shape=(latent_space,))
    siamese_tower_1 = correction(siamese_in_1)
    siamese_tower_2 = correction(siamese_in_2)
    x = layers.Lambda(distance, output_shape=(1,))([siamese_tower_1, siamese_tower_2])
    x = layers.BatchNormalization()(x)
    siamese_out = layers.Dense(1, activation="sigmoid")(x)
    siamese = models.Model([siamese_in_1, siamese_in_2], siamese_out, name='siamese')
    siamese.compile(optimizer=optimizers.RMSprop(), loss=loss(), metrics=['accuracy'])

    history_siamese = siamese.fit(gen_train_pair, validation_data=gen_val_pair,
                                epochs=25, batch_size=batch_train_pairs,
                                steps_per_epoch=batch_train_pairs_steps, validation_steps=batch_val_pairs_steps)
    custom_models.save_model_and_history(correction, history_siamese)

### Risultati


In [ ]:
custom_models.save_model_and_history(correction, history_siamese)
history = [custom_models.load_history_of(correction)]
plots.plot_history(history, ['loss', 'accuracy'])

In [ ]:
from sklearn.decomposition import PCA as dim_reduction

# Get sample batch
indices = np.random.choice(len(x_test), 1000, replace=False)
embedded_true = x_test_encoded[indices]
embedded_new = correction.predict(embedded_true, verbose=0)
labels_true = y_test[indices]
labels_pred = np.argmax(y_test_pred[indices], axis=1)
labels_new = np.argmax(classifier.predict(embedded_new, verbose=0), axis=1)

# Get low-dimensional t-SNE Embeddings
h_embedded_true = dim_reduction(n_components=2).fit_transform(embedded_true)
h_embedded_new = dim_reduction(n_components=2).fit_transform(embedded_new)

# Plot
colors = list(plt.cm.tab10.colors[:total_classes])
colors_true = [colors[i] for i in labels_true]
colors_pred = [colors[i] for i in labels_pred]
colors_new = [colors[i] for i in labels_new]

spaces = [
    (h_embedded_true, colors_true, 'Original Space'),
    (h_embedded_new, colors_true, 'New Space'),
    (h_embedded_true, colors_pred, 'Predicted Space'),
    (h_embedded_new, colors_new, 'Predicted New Space')
]

plt.figure(figsize=(12, 6))
for i, (h, colors, title) in enumerate(spaces):
    plt.subplot(2, 2, i + 1)
    plt.scatter(h[:,0], h[:,1], alpha=0.5, c=colors)
    plt.title(title)
plt.show()

In [ ]:
plots.plot_confusion_matrix([
    ("Training Data", y_train, y_train_pred),
    ("Training Data (Corrected)", y_train, classifier.predict(correction.predict(x_train_encoded, verbose=0), verbose=0))
], total_classes)
plots.plot_confusion_matrix([
    ("Validation Data", y_val, y_val_pred),
    ("Validation Data (Corrected)", y_val, classifier.predict(correction.predict(x_val_encoded, verbose=0), verbose=0))
], total_classes)
plots.plot_confusion_matrix([
    ("Test Data", y_test, y_test_pred),
    ("Test Data (Corrected)", y_test, classifier.predict(correction.predict(x_test_encoded, verbose=0), verbose=0))
], total_classes)